In [46]:
!pip3 install -r requirements.txt

DEPRECATION: qiskit-nature 0.5.1 has a non-standard dependency specifier qiskit-terra>=0.22.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of qiskit-nature or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [51]:
from requests.auth import HTTPBasicAuth
from ibm_cloud_sdk_core.api_exception import ApiException
from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import json
from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
import DiscoveryDetails as dt


from pprint import pprint

- get the api key and url from the resources list page
- https://cloud.ibm.com/services/discovery/crn%3Av1%3Abluemix%3Apublic%3Adiscovery%3Aus-south%3Aa%2F46c4f2c9bc554c8293a3574f4d0bee64%3A550014a0-5903-4e54-9704-42e2d5847a42%3A%3A?paneId=manage
- project id from the projects -> integrate and deploy --> API Information

In [52]:
#cloud_apikey = 'yejHZ5NsoHKD37gncG4dOUZlMbmKA4gWrnvNRMPpaLu2'
apikey = 'upuzegAg8qE2aHwOob6ai-r9uENdUeA-5ZUExvy-5Juc' # from the WD resource list https://cloud.ibm.com/services/discovery/crn%3Av1%3Abluemix%3Apublic%3Adiscovery%3Aus-south%3Aa%2F46c4f2c9bc554c8293a3574f4d0bee64%3A550014a0-5903-4e54-9704-42e2d5847a42%3A%3A?paneId=manage
project_id = 'acfe53ea-a6e5-4c1d-a05c-352a145a4ca3'
#collection_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxx'
#url = 'https://api.xxxx.discovery.watson.cloud.ibm.com/instances/xxxxxxxxxxxx'
url = 'https://api.us-south.discovery.watson.cloud.ibm.com/instances/550014a0-5903-4e54-9704-42e2d5847a42'
version = '2020-08-30'


authenticator = IAMAuthenticator(apikey)
discovery = DiscoveryV2(
    version=version,
    authenticator=authenticator
)


discovery.set_service_url(url)#('https://us-south.discovery.watson.cloud.ibm.com')
discovery.set_disable_ssl_verification(True)

#https://us-south.discovery.watson.cloud.ibm.com/v2/instances/crn%3Av1%3Abluemix%3Apublic%3Adiscovery%3Aus-south%3Aa%2F46c4f2c9bc554c8293a3574f4d0bee64%3A550014a0-5903-4e54-9704-42e2d5847a42%3A%3A/
#projects/ec800c2a-ecc1-49ca-a3fd-f7a9addf2e43/workspace/preview/

In [136]:

response = discovery.list_collections(
  project_id = project_id
).get_result()


/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [137]:
print(json.dumps(response, indent=2))
#pprint(response)

{
  "collections": [
    {
      "name": "RelTrain Collection 2",
      "collection_id": "9304a6de-75dc-bc3f-0000-0191deb87ae0"
    }
  ]
}


In [138]:
collection_id = response['collections'][0]['collection_id']

In [139]:
collection_id

'9304a6de-75dc-bc3f-0000-0191deb87ae0'

In [140]:
Question = 'What are the list of illnesses and the exclusions from a critical illness cover?'
result = discovery.query(
                project_id=project_id,
                query=question,
                count=5
            ).get_result()

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [141]:
result

{'matching_results': 1,
 'retrieval_details': {'document_retrieval_strategy': 'untrained'},
 'aggregations': [{'type': 'term',
   'field': 'enriched_text.entities.text',
   'name': 'entities',
   'results': [{'key': '$2.50', 'matching_results': 1},
    {'key': '$25,000', 'matching_results': 1},
    {'key': '$5,000', 'matching_results': 1},
    {'key': '$50,000', 'matching_results': 1},
    {'key': '1', 'matching_results': 1},
    {'key': '1-844-528-0583', 'matching_results': 1},
    {'key': '1-877-893-9893', 'matching_results': 1},
    {'key': '1.0', 'matching_results': 1},
    {'key': '10', 'matching_results': 1},
    {'key': '100 \nmilliliters', 'matching_results': 1}]}],
 'results': [{'document_id': '04711489-3a1a-4cf9-a8b1-026003bdb83a',
   'result_metadata': {'collection_id': '9304a6de-75dc-bc3f-0000-0191deb87ae0'},
   'enriched_text': [{'entities': [{'model_name': 'natural_language_understanding',
       'mentions': [{'confidence': 0.3750491,
         'location': {'end': 52, 'beg

In [176]:
## Add document

def upload_document (file_path):
    with open(file_path, "rb") as file:
        add_doc_response = discovery.add_document(
            project_id=project_id,
            collection_id=collection_id, 
            file=file, 
            file_content_type='application/pdf',
            metadata='{"domain":""}'
       ).get_result()
        print (add_doc_response)
        # Getting the ID of the newly added document
        new_document_id = add_doc_response.get("document_id", None)
        
        if new_document_id:
            print(f"Newly added document ID: {new_document_id}")
        else:
            print("Error getting document ID.")

In [177]:
#file_path = "/Users/hema/feb_2022/WS/HQ/RelevancyTraining/express-ci-guide-to-definitions.pdf"  #w/ metadata
file_path = "/Users/hema/feb_2022/WS/HQ/RelevancyTraining/Sample-Brochure.pdf" #w/o metadata
    
upload_doc_discovery = upload_document(file_path)

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'document_id': 'aafe9924-691b-4864-83d1-86dd06de4976', 'status': 'pending'}
Newly added document ID: aafe9924-691b-4864-83d1-86dd06de4976


In [178]:
##list documents
response = discovery.list_documents(
   project_id=project_id,
   collection_id=collection_id, 
  ).get_result()
response

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'matching_results': 4,
 'documents': [{'document_id': '9d6081619f893d57bb85495e78e9303a'},
  {'document_id': '04711489-3a1a-4cf9-a8b1-026003bdb83a'},
  {'document_id': '05396343-056c-44d8-bf8d-4f26971ce901'},
  {'document_id': 'aafe9924-691b-4864-83d1-86dd06de4976'}]}

In [99]:
response = discovery.query(
  project_id=project_id,
  query="enriched_text.entities.text:Teladoc Medical Experts,extracted_metadata.filename:express-ci-guide-to-definitions.pdf"
).get_result()


#"extracted_metadata.filename:express-ci-guide-to-definitions.pdf",

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [100]:
response

{'matching_results': 4,
 'retrieval_details': {'document_retrieval_strategy': 'untrained'},
 'aggregations': [{'type': 'term',
   'field': 'enriched_text.entities.text',
   'name': 'entities',
   'results': [{'key': 'first', 'matching_results': 3},
    {'key': '$2.50', 'matching_results': 2},
    {'key': '$5,000', 'matching_results': 2},
    {'key': '5', 'matching_results': 2},
    {'key': '5 years', 'matching_results': 2},
    {'key': 'SUN LIFE', 'matching_results': 2},
    {'key': 'Teladoc Medical', 'matching_results': 2},
    {'key': 'Teladoc Medical Experts', 'matching_results': 2},
    {'key': 'expert', 'matching_results': 2},
    {'key': 'month', 'matching_results': 2}]}],
 'results': [{'document_id': 'a5b52520-e90e-4a3b-a6ed-d2a763c04a91_9',
   'result_metadata': {'collection_id': 'b570c01a-8576-481a-0000-0191c7a21660'},
   'enriched_text': [{'entities': [{'model_name': 'natural_language_understanding',
       'mentions': [{'confidence': 0.88706017,
         'location': {'end': 

In [142]:
response = discovery.query(
  project_id=project_id,
  query="enriched_text.entities.text:Teladoc Medical Experts,metadata.domain:health_insurance"
).get_result()

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [143]:
response

{'matching_results': 1,
 'retrieval_details': {'document_retrieval_strategy': 'untrained'},
 'aggregations': [{'type': 'term',
   'field': 'enriched_text.entities.text',
   'name': 'entities',
   'results': [{'key': '$2.50', 'matching_results': 1},
    {'key': '$25,000', 'matching_results': 1},
    {'key': '$5,000', 'matching_results': 1},
    {'key': '$50,000', 'matching_results': 1},
    {'key': '1', 'matching_results': 1},
    {'key': '1-844-528-0583', 'matching_results': 1},
    {'key': '1-877-893-9893', 'matching_results': 1},
    {'key': '1.0', 'matching_results': 1},
    {'key': '10', 'matching_results': 1},
    {'key': '100 \nmilliliters', 'matching_results': 1}]}],
 'results': [{'document_id': '04711489-3a1a-4cf9-a8b1-026003bdb83a',
   'result_metadata': {'collection_id': '9304a6de-75dc-bc3f-0000-0191deb87ae0'},
   'enriched_text': [{'entities': [{'model_name': 'natural_language_understanding',
       'mentions': [{'confidence': 0.3750491,
         'location': {'end': 52, 'beg

In [189]:
response = discovery.query(
  project_id=project_id,
  query='illness coverage,metadata.domain:health'
).get_result()

/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [190]:
response

{'matching_results': 1,
 'retrieval_details': {'document_retrieval_strategy': 'untrained'},
 'aggregations': [{'type': 'term',
   'field': 'enriched_text.entities.text',
   'name': 'entities',
   'results': [{'key': '$2.50', 'matching_results': 1},
    {'key': '$25,000', 'matching_results': 1},
    {'key': '$5,000', 'matching_results': 1},
    {'key': '$50,000', 'matching_results': 1},
    {'key': '1', 'matching_results': 1},
    {'key': '1-844-528-0583', 'matching_results': 1},
    {'key': '1-877-893-9893', 'matching_results': 1},
    {'key': '1.0', 'matching_results': 1},
    {'key': '10', 'matching_results': 1},
    {'key': '100 \nmilliliters', 'matching_results': 1}]}],
 'results': [{'document_id': '04711489-3a1a-4cf9-a8b1-026003bdb83a',
   'result_metadata': {'collection_id': '9304a6de-75dc-bc3f-0000-0191deb87ae0'},
   'enriched_text': [{'entities': [{'model_name': 'natural_language_understanding',
       'mentions': [{'confidence': 0.3750491,
         'location': {'end': 52, 'beg

In [58]:
output_file = open("./training_file.tsv", "w")
writer = csv.writer(output_file, delimiter="\t")
try:
    project_id = 'acfe53ea-a6e5-4c1d-a05c-352a145a4ca3'

    with open("./Questions.txt", encoding="Windows 1252") as questions:
        noOfQuestions = 0
        for line in questions:
            print("Question No = " + str(noOfQuestions + 1))
            question = line.replace("\n", "")
            print("Question = " + question)

            question = "%s" % (question)

            # run Discovery query to get results from untrained service
            result = discovery.query(
                project_id=project_id,
                query=question,
                count=5
            ).get_result()

            #print("Query Response = " + json.dumps(result))

            # create a row for each query and results
            result_list = [question]
            for resultDoc in result["results"]:
                id = resultDoc["document_id"]
                text = resultDoc["text"]
                if(len(text) > 1000):
                    text = text[:1000]
                # leave a space to enter a relevance label for each doc
                result_list.extend([id, text, ' '])

            # write the row to the file
            writer.writerow(result_list)
            noOfQuestions = noOfQuestions + 1
            print("==========================================================")
            print("")
    print("tsv file with questions and query results created")
    print("Number of questions processed = " + str(noOfQuestions))
    output_file.close()
except Exception as e:
    print("Exception occurred ####### ")
    print(e)


Question No = 1
Question = What are the list of illnesses and the exclusions from a critical illness cover?


/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Question No = 2
Question = What are the 34 illnesses covered under the plan?


/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Question No = 3
Question = Do you cover angioplasty and other heart related disease?


...


Question No = 139
Question = When can I claim critical illness benefit?


/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Question No = 140
Question = When can I get critical illness pay-out?


/Users/hema/opt/anaconda3/envs/DND/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.discovery.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



tsv file with questions and query results created
Number of questions processed = 140
